<a href="https://colab.research.google.com/github/Annie-Yeeun-Jang/Info_retrieval_project/blob/master/%5BSI650_project%5D_MF_leave_one_out.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.sparse.linalg import svds

from google.colab import drive
drive.mount('/content/gdrive')

import os
os.chdir('/content/gdrive/MyDrive/650/team_proj')

Mounted at /content/gdrive


# Preprocessing

In [ ]:
jester=pd.read_csv('jester.csv') #ratings matrix with nan and range -10~+10 

In [ ]:
jester

,1,2,3,4,5,6,7,8,9,10,...,141,142,143,144,145,146,147,148,149,150
0,NaN,NaN,NaN,NaN,0.21875,NaN,-9.28125,-9.28125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,-9.68750,NaN,9.93750,9.53125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,-9.84375,NaN,-9.84375,-7.21875,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,6.90625,NaN,4.75000,-5.90625,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,-0.03125,NaN,-9.09375,-0.40625,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,NaN,NaN,NaN,NaN,NaN,NaN,-6.53125,-2.34375,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54901,NaN,NaN,NaN,NaN,NaN,NaN,8.93750,9.78125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54902,NaN,NaN,NaN,NaN,NaN,NaN,-1.59375,4.53125,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54903,NaN,NaN,NaN,NaN,NaN,NaN,-7.40625,6.93750,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
jester.notnull().sum()[:20] #they are some jokes that nobody rated [1,2,3,4,6,9,10,11,12,14]

1         0
2         0
3         0
4         0
5       566
6         0
7     54905
8     54905
9         0
10        0
11        0
12        0
13    54905
14        0
15    54905
16    54905
17    54905
18    54905
19    54905
20      557
dtype: int64

### we need to do some preprocessing (drop users only having negative ratings or few ratings(<5), jokes that have never been rated)

In [ ]:
#dropping jokes that have never been rated
proc=jester.drop(columns=['1','2','3','4','6','9','10','11','12','14'], axis=1)
proc

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-9.84375,-9.84375,-7.21875,-2.03125,-9.93750,-9.96875,-9.87500,-9.81250,-9.78125,-6.84375,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54901,NaN,8.93750,9.78125,9.53125,9.50000,9.93750,9.87500,9.96875,9.96875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54902,NaN,-1.59375,4.53125,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54903,NaN,-7.40625,6.93750,-8.00000,-5.40625,-0.03125,5.15625,-3.62500,4.15625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
proc.notnull().sum().min()

166

In [ ]:
proc.max(axis=1)

0        9.90625
1        9.93750
2        1.25000
3        8.00000
4        8.40625
          ...   
54900    2.34375
54901    9.96875
54902    4.53125
54903    6.93750
54904    9.31250
Length: 54905, dtype: float64

In [ ]:
proc.values

array([[ 0.21875, -9.28125, -9.28125, ...,      nan,      nan,      nan],
       [-9.6875 ,  9.9375 ,  9.53125, ...,      nan,      nan,      nan],
       [-9.84375, -9.84375, -7.21875, ...,      nan,      nan,      nan],
       ...,
       [     nan, -1.59375,  4.53125, ...,      nan,      nan,      nan],
       [     nan, -7.40625,  6.9375 , ...,      nan,      nan,      nan],
       [     nan,  4.25   ,  6.59375, ...,      nan,      nan,      nan]])

In [ ]:
#how to change it to 0-1 rating ; https://stackoverflow.com/questions/19766757/replacing-numpy-elements-if-condition-is-met
proc.values

array([[ 0.21875, -9.28125, -9.28125, ...,      nan,      nan,      nan],
       [-9.6875 ,  9.9375 ,  9.53125, ...,      nan,      nan,      nan],
       [-9.84375, -9.84375, -7.21875, ...,      nan,      nan,      nan],
       ...,
       [     nan, -1.59375,  4.53125, ...,      nan,      nan,      nan],
       [     nan, -7.40625,  6.9375 , ...,      nan,      nan,      nan],
       [     nan,  4.25   ,  6.59375, ...,      nan,      nan,      nan]])

In [ ]:
#0-1 conversion
zero_one_mat=pd.DataFrame((proc.values > 0).astype(int), index=proc.index, columns=proc.columns)
zero_one_mat

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54900,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54901,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
54902,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54903,0,0,1,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
zero_one_mat.sum(axis=1)

0        43
1        24
2         3
3        74
4        10
         ..
54900     5
54901     8
54902     8
54903     3
54904    12
Length: 54905, dtype: int64

In [ ]:
np.array(np.where(zero_one_mat.sum(axis=1) <5)).shape

(1, 10104)

In [ ]:
zero_one_mat=zero_one_mat.iloc[np.where(zero_one_mat.sum(axis=1) >4)]
zero_one_mat # matrix only having users rated >4 jokes

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54899,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54900,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
54901,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
54902,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
zero_one_mat.to_pickle('zero_one_mat_filtered.pkl')

In [ ]:
good_users_list=zero_one_mat.index

In [ ]:
good_users_list

Int64Index([    0,     1,     3,     4,     5,     6,     7,     8,     9,
               10,
            ...
            54893, 54894, 54895, 54896, 54897, 54899, 54900, 54901, 54902,
            54904],
           dtype='int64', length=44801)

In [ ]:
proc_filtered=proc.iloc[good_users_list]
proc_filtered #numeric ratings only contining good users 

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,-2.90625,-2.34375,-0.50000,-0.96875,2.25000,0.40625,0.87500,0.40625,1.43750,-0.03125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54899,NaN,3.62500,-8.40625,-7.40625,5.28125,-8.93750,3.12500,-7.09375,-9.40625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54900,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54901,NaN,8.93750,9.78125,9.53125,9.50000,9.93750,9.87500,9.96875,9.96875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54902,NaN,-1.59375,4.53125,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
proc_filtered.max(axis=1).min() #negative users are gone

0.0625

In [ ]:
proc_filtered.to_pickle('filtered_rating_mat.pkl') #should I reset the user idx?(think it's better to reset)

## Leave one out test split

In [ ]:
#cut off by certain threshold (rating >5)
arr=np.array(jester.iloc[0])
thres=5
np.array(np.where(arr > thres))+1 #indices of elements with positive ratings

array([[ 27,  29,  31,  32,  35,  36,  50,  53,  54,  65,  66,  69,  72,
         76,  83,  87,  89,  91,  93, 108, 109, 119, 120, 121, 123, 127]])

In [ ]:
train_matrix=jester.copy()

In [ ]:
row=2
arr=jester.iloc[row].fillna(-1000)
top_k_idx=np.argsort(arr)
top_k_idx

1        0
2       94
3       95
4       96
5       97
      ... 
146     12
147     38
148     37
149    101
150    104
Name: 2, Length: 150, dtype: int64

In [ ]:
np.array(top_k_idx[::-1][:10])

array([104, 101,  37,  38,  12,  19,   7,  18,  33,  32])

In [ ]:
train_matrix.iloc[row,104]

1.25

In [ ]:
jester.max(axis=1).min() #wtf such a negative user

-10.0

In [ ]:
true_cols=proc_filtered.columns[1:]
true_cols

Index(['5', '7', '8', '13', '15', '16', '17', '18', '19', '20',
       ...
       '141', '142', '143', '144', '145', '146', '147', '148', '149', '150'],
      dtype='object', length=140)

In [ ]:
proc_filtered=proc_filtered.reset_index()

In [ ]:
proc_filtered.drop(columns=['index'],inplace=True)

In [ ]:
proc_filtered.columns=range(0,140)

In [ ]:
train_matrix=proc_filtered.copy()

In [ ]:
train_matrix

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-2.90625,-2.34375,-0.50000,-0.96875,2.25000,0.40625,0.87500,0.40625,1.43750,-0.03125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,NaN,3.62500,-8.40625,-7.40625,5.28125,-8.93750,3.12500,-7.09375,-9.40625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44797,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44798,NaN,8.93750,9.78125,9.53125,9.50000,9.93750,9.87500,9.96875,9.96875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44799,NaN,-1.59375,4.53125,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#ndcg: just row-to-row comparision btw gt and pred with rows full of 0,1s
#code seems okay(need some sanity checks), but we should filter out some inappropriate users/jokes
#do some sanity checks
import random
k=5
test_jokes=[]
for row in proc_filtered.index:
  #arr=np.array(jester.iloc[row].dropna())
  arr=proc_filtered.iloc[row].fillna(-1000)
  top_k_idx=np.array(np.argsort(arr)[::-1][:k]) #top k indices
  #print(top_k_idx)
  chosen_test=random.choice(top_k_idx)
  test_jokes.append(chosen_test)
  #print(train_matrix.iloc[row,chosen_test])
  train_matrix.iloc[row,chosen_test]=np.nan

In [ ]:
row

44800

Think it works well!!

In [ ]:
train_matrix

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-2.90625,-2.34375,-0.50000,-0.96875,2.25000,0.40625,0.87500,0.40625,1.43750,-0.03125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,NaN,NaN,-8.40625,-7.40625,5.28125,-8.93750,3.12500,-7.09375,-9.40625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44797,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44798,NaN,8.93750,9.78125,9.53125,9.50000,NaN,9.87500,9.96875,9.96875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
44799,NaN,-1.59375,NaN,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
(train_matrix.isnull().sum(axis=1) - proc_filtered.isnull().sum(axis=1)).sum() #sanity check, should be equal to num of rows

44801

In [ ]:
train_matrix.columns=true_cols

In [ ]:
train_matrix.to_pickle('masked_for_train.pkl')

In [ ]:
proc_filtered.columns=true_cols

In [ ]:
proc_filtered.to_pickle('gt_matrix.pkl')

In [ ]:
test_jokes

In [ ]:
len(test_jokes)

44801

In [ ]:
true_cols

Index(['5', '7', '8', '13', '15', '16', '17', '18', '19', '20',
       ...
       '141', '142', '143', '144', '145', '146', '147', '148', '149', '150'],
      dtype='object', length=140)

In [ ]:
proc_filtered.columns

RangeIndex(start=0, stop=140, step=1)

In [ ]:
columns_map_dict=dict(zip(proc_filtered.columns, true_cols))

In [ ]:
test_jokes_mapped=[columns_map_dict[k] for k in test_jokes]

In [ ]:
len(test_jokes_mapped)

44801

In [ ]:
proc_filtered['test_joke_idx']=test_jokes_mapped

In [ ]:
proc_filtered.to_pickle('gt_matrix.pkl')

In [ ]:
proc_filtered

,5,7,8,13,15,16,17,18,19,20,...,142,143,144,145,146,147,148,149,150,test_joke_idx
0,0.21875,-9.28125,-9.28125,-6.78125,0.87500,-9.65625,-9.03125,-7.46875,-8.71875,-9.15625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109
1,-9.68750,9.93750,9.53125,9.93750,0.40625,3.71875,9.65625,-2.68750,-9.56250,-9.12500,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50
2,6.90625,4.75000,-5.90625,-0.40625,-4.03125,3.87500,6.21875,5.65625,6.09375,5.40625,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22
3,-0.03125,-9.09375,-0.40625,7.50000,-7.21875,-9.43750,0.12500,-9.15625,3.65625,-9.43750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21
4,-2.90625,-2.34375,-0.50000,-0.96875,2.25000,0.40625,0.87500,0.40625,1.43750,-0.03125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,93
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,NaN,3.62500,-8.40625,-7.40625,5.28125,-8.93750,3.12500,-7.09375,-9.40625,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7
44797,NaN,-6.53125,-2.34375,1.43750,-4.37500,-9.90625,-4.06250,-3.93750,-9.00000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69
44798,NaN,8.93750,9.78125,9.53125,9.50000,9.93750,9.87500,9.96875,9.96875,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16
44799,NaN,-1.59375,4.53125,-9.21875,-3.09375,-1.96875,1.12500,-5.93750,-4.34375,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8


In [ ]:
test_rating=[]
for i in range(44801):
  idx=str(proc_filtered.loc[i,'test_joke_idx'])
  test_rating.append(proc_filtered.loc[i,idx])

In [ ]:
min(test_rating)

0.03125

# 0-1 conversion on Train, test data

In [ ]:
gt_matrix=pd.read_pickle('gt_matrix.pkl')
train_matrix=pd.read_pickle('masked_for_train.pkl')

In [ ]:
zero_one_train=pd.DataFrame((train_matrix.values > 0).astype(int), index=train_matrix.index, columns=train_matrix.columns)
zero_one_train

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44797,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44798,0,1,1,1,1,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
44799,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
gt_matrix2=gt_matrix.fillna(-30).drop(['test_joke_idx'], axis=1)
zero_one_gt=pd.DataFrame((gt_matrix2.values > 0).astype(int), index=gt_matrix2.index, columns=gt_matrix2.columns)
zero_one_gt

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,0,1,0,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44797,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
44798,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
44799,0,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
np.array(np.where(zero_one_train.sum(axis=1) <4))#sanity check => users rated less than 4 jokes => should not exist bc we already filtered out

array([], shape=(1, 0), dtype=int64)

# SVD

In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
zero_one_train=zero_one_train.astype(float)

In [ ]:
U, sigma, Vt = svds(zero_one_train, k = 6)
print(U.shape)
print(sigma.shape)
print(Vt.shape)

(44801, 6)
(6,)
(6, 140)


In [ ]:
sigma = np.diag(sigma)
sigma

array([[121.08043857,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        , 127.3041421 ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        , 155.96665679,   0.        ,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , 188.64989456,
          0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        273.27564392,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
          0.        , 731.04073051]])

In [ ]:
svd_user_predicted_link = np.dot(np.dot(U, sigma), Vt) 

In [ ]:
restored=pd.DataFrame(svd_user_predicted_link, columns=zero_one_train.columns)
restored

,5,7,8,13,15,16,17,18,19,20,...,141,142,143,144,145,146,147,148,149,150
0,0.000721,0.048119,0.099257,0.161007,0.042419,0.053831,0.360088,0.177038,0.317256,-0.000440,...,-0.020243,0.188006,0.504445,0.258440,0.425396,0.003919,0.245771,0.650885,0.397398,0.331374
1,0.007845,0.696686,0.776212,0.807177,0.721625,0.686477,0.826959,0.798396,0.790546,0.009385,...,0.006142,0.052657,0.060986,0.108445,0.030950,0.011469,0.055518,0.228224,0.116046,0.090580
2,0.011905,0.612690,0.739846,0.699353,0.655316,0.612399,0.693341,0.714132,0.702861,0.013295,...,0.201060,0.468966,0.386512,0.513975,0.383746,0.367951,0.535591,0.510706,0.537729,0.551042
3,0.003475,0.325704,0.365715,0.410304,0.339442,0.350857,0.520385,0.417519,0.465991,0.004132,...,-0.027175,0.031007,0.056158,0.033918,0.033093,-0.041926,-0.011460,0.167758,0.061546,-0.084356
4,0.008553,0.646599,0.705430,0.759418,0.664291,0.615820,0.752086,0.728888,0.699244,0.009552,...,0.025131,0.108217,0.264384,0.240525,0.210758,0.056190,0.183763,0.411070,0.257520,0.180411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44796,0.002421,0.240351,0.263411,0.270440,0.257266,0.252592,0.268848,0.286960,0.254404,0.003087,...,-0.000502,-0.012783,0.047780,0.016334,0.031731,-0.004021,-0.006640,0.001891,-0.015297,-0.003638
44797,0.001178,0.132230,0.129398,0.153774,0.146988,0.149430,0.150232,0.175644,0.121101,0.001403,...,-0.014602,-0.041700,0.171257,0.031737,0.134242,-0.024140,0.000969,0.023474,-0.015034,-0.027195
44798,0.007432,0.752992,0.859827,0.850195,0.808177,0.810245,0.908924,0.913762,0.885566,0.009898,...,0.011164,0.013695,-0.062148,-0.012983,-0.070765,0.003895,-0.062609,-0.072578,-0.076383,-0.038039
44799,-0.000358,0.113302,0.158465,0.159022,0.126576,0.135313,0.234504,0.208433,0.240072,-0.000365,...,-0.025280,0.003339,0.172602,0.021120,0.133191,-0.037140,0.016800,0.119818,0.038522,0.168044


# Evaluation

In [ ]:
from sklearn.metrics import ndcg_score

In [ ]:
user_id=0
ndcg_score(np.asarray([zero_one_gt.loc[user_id]]), np.asarray([restored.loc[user_id]]))

0.8900517399360979

In [ ]:
arr=np.array([4,5,1,3])
arr.argsort()[::-1].argsort()+1

array([2, 1, 4, 3])

In [ ]:
def get_rank(arr):
  return arr.argsort()[::-1].argsort()+1

In [ ]:
gt_rank=get_rank(zero_one_gt.loc[user_id])
pred_rank=get_rank(restored.loc[user_id])

In [ ]:
np.asarray([gt_rank])

array([[ 37,  70,  71,  72,  22,  74,  75,  76,  77,  78,  79,  80,  81,
         82,  83,  23,  24,  86,  25,  88,  26,  27,  67,  68,  21,  28,
         46,  47,  48,  49,  50,  30,  52,  45,  54,  53,  56,  57,  31,
         32,  33,  34,  35,  29,  63,  64,  65,  91,  69,  93,  20,  19,
        121, 122,  18,   3, 125, 126,   4, 128, 119,   5, 131, 132, 133,
          6, 134, 135, 136,   1,   7, 139,   8, 118, 117, 116,   9,  96,
         10,  97,  11,  12,  13,  99, 100, 101, 102, 103, 104, 105, 106,
         14, 107,  15,  16, 110,  17,  36,   2, 114, 115, 120,  44, 113,
        112, 111, 109, 108,  42,  41,  40,  98,  39,  95, 130, 138,  43,
        129, 127, 124, 123,  92,  94,  38,  62,  61,  60,  59,  58,  51,
         66,  55,  90,  89,  87,  85,  84,  73, 137, 140]])

In [ ]:
ndcg_score(np.asarray([gt_rank]), np.asarray([pred_rank]), k=10)

0.6501172070334261

In [ ]:
ndcg_cum=0
for user_id in range(len(zero_one_gt)):
  gt_rank=get_rank(zero_one_gt.loc[user_id])
  pred_rank=get_rank(restored.loc[user_id])
  ndcg_cum+=ndcg_score(np.asarray([gt_rank]), np.asarray([pred_rank]), k=5)

In [ ]:
ndcg_total=ndcg_cum/len(zero_one_gt)
ndcg_total #for k=5

0.5858030393185563

In [ ]:
ndcg_total=ndcg_cum/len(zero_one_gt)
ndcg_total #for k=10

0.6005982468584248

test data 따로 랜덤하게 뽑아갖고 hit ratio는 수정해야 할듯

In [ ]:
hit=0
k=5
for user_id in range(len(zero_one_gt)):
  top_k_pred=list(restored.loc[user_id].sort_values(ascending=False).index[:k])
  #top_true=zero_one_gt.loc[user_id].sort_values(ascending=False).index[0] #이부분 코드 수정 !! (row별로 test idx 뽑아와서 소팅 없이)
  top_true=gt_matrix.loc[user_id,'test_joke_idx']
  if top_true in top_k_pred:
    hit+=1

In [ ]:
hit_ratio=hit/len(zero_one_gt)
hit_ratio #for k=5

0.20829892189906477

In [ ]:
hit_ratio=hit/len(zero_one_gt)
hit_ratio #for k=10

0.3839646436463472

## Naive approach evaluation

In [ ]:
naive_pred=df_processed.mean()
naive_pred

1       0.000000
2       0.000000
3       0.000000
4       0.000000
5       0.288287
         ...    
146     2.824381
147     5.914830
148     9.696221
149     6.799298
150    11.493085
Length: 150, dtype: float64

In [ ]:
#ndcg
ndcg_cum=0
naive_rank=get_rank(naive_pred)
for user_id in range(len(df_processed)):
  gt_rank=get_rank(df_processed.loc[user_id])
  ndcg_cum+=ndcg_score(np.asarray([gt_rank]), np.asarray([naive_rank]), k=10)

In [ ]:
ndcg_total=ndcg_cum/len(df_processed)
ndcg_total #for k=5

0.6135019662129706

In [ ]:
ndcg_total=ndcg_cum/len(df_processed)
ndcg_total #for k=10

0.5629028558834749

In [ ]:
#hit ratio
hit=0
k=10
naive_top_k_pred=list(naive_pred.sort_values(ascending=False).index[:k])
for user_id in range(len(df_processed)):
  top_true=df_processed.loc[user_id].sort_values(ascending=False).index[0]
  if top_true in naive_top_k_pred:
    hit+=1

In [ ]:
hit_ratio=hit/len(df_processed)
hit_ratio #for k=5

0.260304161733904

In [ ]:
hit_ratio=hit/len(df_processed)
hit_ratio #for k=10

0.44050632911392407

## Below here is just for reference
- Codes we can refer for more advanced models:
- https://github.com/microsoft/recommenders
- https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/lightgcn_deep_dive.ipynb
- https://github.com/microsoft/recommenders/blob/main/examples/02_model_collaborative_filtering/standard_vae_deep_dive.ipynb


In [ ]:
df_svd_preds = pd.DataFrame(svd_user_predicted_link,index= ply_pivot1.index ,columns = ply_pivot1.columns)
df_svd_preds

In [ ]:
#우리가 캡스톤에 사용했던 방식은 결국 leave one out인거암. 
def recommend_songs(df_svd_preds, user_id, ori_link_df, num_recommendations):
    # 최종적으로 만든 pred_df에서 사용자 index에 따라 곡 데이터 정렬 -> 영화 유사도가 높은 순으로 정렬 됌
    sorted_user_predictions = df_svd_preds.loc[user_id].sort_values(ascending=False)
    # 원본 곡 데이터에서 user id에 해당하는 데이터를 뽑아낸다. 
    #user_data = ori_link_df.loc[user_id]
    # 사용자의 곡 유사도가 높은 순으로 정렬된 데이터와 위 recommendations을 합친다. 
    recommendations = pd.DataFrame(sorted_user_predictions).reset_index()
    # 컬럼 이름 바꾸고 정렬해서 return
    recommendations = recommendations.rename(columns = {user_id: 'Predictions'}).sort_values('Predictions', ascending = False).iloc[:num_recommendations, :]
    return recommendations

In [ ]:
predictions = recommend_songs(df_svd_preds, 23431, ply_pivot1, 100)

In [ ]:
predictions

In [ ]:
import json
 
# Ready for data
songs = []
song = {}
for name in ply_pivot_val.id:
  song['id'] = name
  song['songs'] = list(recommend_songs(df_svd_preds, name, ply_pivot1, 100)['song_id'])
  songs.append(song)

 
# Print JSON
print(json.dumps(songs, indent="\t") )

In [ ]:
# Write JSON
with open('./답안/res.json', 'w', encoding="utf-8") as make_file:
    json.dump(songs, make_file, ensure_ascii=False, indent="\t")

In [ ]:
f = open('./답안/res.json', encoding="UTF-8")
raw_data = json.loads(f.read())
raw_data